In [1]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 645 kB/s 


In [2]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv(r'/content/gdrive/My Drive/drugEffects.csv')
df.head()

,Unnamed: 0,urlDrugName,rating,effectiveness,condition,sideEffects
0,0,enalapril,4,Highly Effective,management of congestive heart failure,Mild Side Effects
1,1,ortho-tri-cyclen,1,Highly Effective,birth prevention,Severe Side Effects
2,2,ponstel,10,Highly Effective,menstrual cramps,No Side Effects
3,3,prilosec,3,Marginally Effective,acid reflux,Mild Side Effects
4,4,lyrica,2,Marginally Effective,fibromyalgia,Severe Side Effects


In [ ]:
df=df.dropna()

In [ ]:
df=df.drop('Unnamed: 0',axis=1)

In [ ]:
X = df.drop('sideEffects',axis=1)
y = df['sideEffects']
X.head()

,urlDrugName,rating,effectiveness,condition
0,enalapril,4,Highly Effective,management of congestive heart failure
1,ortho-tri-cyclen,1,Highly Effective,birth prevention
2,ponstel,10,Highly Effective,menstrual cramps
3,prilosec,3,Marginally Effective,acid reflux
4,lyrica,2,Marginally Effective,fibromyalgia


In [ ]:
encoderx = ce.BinaryEncoder(cols=X)
Xn = encoderx.fit_transform(X)
Xn.head()

,urlDrugName_0,urlDrugName_1,urlDrugName_2,urlDrugName_3,urlDrugName_4,urlDrugName_5,urlDrugName_6,urlDrugName_7,urlDrugName_8,urlDrugName_9,...,condition_2,condition_3,condition_4,condition_5,condition_6,condition_7,condition_8,condition_9,condition_10,condition_11
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
encodery = ce.OrdinalEncoder(y)
yn = encodery.fit_transform(y)
yn.head()

,sideEffects
0,1
1,2
2,3
3,1
4,2


In [ ]:
from imblearn.over_sampling import SMOTE


oversample = SMOTE()
Xn, yn = oversample.fit_resample(Xn,yn)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xn, yn, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3413, 31) (1682, 31) (3413, 1) (1682, 1)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train.values.ravel())

RandomForestClassifier()

In [ ]:
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           1       0.45      0.50      0.47       304
           2       0.77      0.74      0.76       335
           3       0.59      0.62      0.61       317
           4       0.95      0.89      0.92       360
           5       0.65      0.62      0.64       366

    accuracy                           0.68      1682
   macro avg       0.68      0.68      0.68      1682
weighted avg       0.69      0.68      0.69      1682



In [ ]:
from sklearn.svm import SVC
svc = SVC(class_weight='balanced')


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[0.001, 0.01, 0.1, 0.5, 1]}
gm = GridSearchCV(svc,param_grid=param_grid)

In [ ]:
gm.fit(X_train, y_train.values.ravel())

GridSearchCV(estimator=SVC(class_weight='balanced'),
             param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1]})

In [ ]:
gm.best_params_

{'C': 1}

In [ ]:
svy = gm.predict(X_test)

In [ ]:
print(classification_report(y_test, svy))

              precision    recall  f1-score   support

           1       0.44      0.44      0.44       341
           2       0.70      0.60      0.65       321
           3       0.63      0.58      0.61       334
           4       0.80      0.93      0.86       338
           5       0.53      0.56      0.55       348

    accuracy                           0.62      1682
   macro avg       0.62      0.62      0.62      1682
weighted avg       0.62      0.62      0.62      1682



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn  = KNeighborsClassifier()
knn.fit(X_train, y_train.values.ravel())

KNeighborsClassifier()

In [ ]:
knny = knn.predict(X_test)

In [ ]:
print(classification_report(knny,y_test))

              precision    recall  f1-score   support

           1       0.39      0.40      0.39       335
           2       0.75      0.63      0.68       381
           3       0.46      0.57      0.51       271
           4       0.94      0.79      0.86       403
           5       0.51      0.61      0.55       292

    accuracy                           0.61      1682
   macro avg       0.61      0.60      0.60      1682
weighted avg       0.64      0.61      0.62      1682



In [ ]:
#add more ml algos and boosting 
#if still not works try hyperparameter tuning

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ada = AdaBoostClassifier(n_estimators=100,learning_rate=1)

In [ ]:
ada.fit(X_train, y_train.values.ravel())

AdaBoostClassifier(learning_rate=1, n_estimators=100)

In [ ]:
a_p = ada.predict(X_test)

In [ ]:
print(classification_report(a_p,y_test))

              precision    recall  f1-score   support

           1       0.35      0.35      0.35       341
           2       0.27      0.33      0.30       263
           3       0.53      0.54      0.53       330
           4       0.68      0.53      0.60       436
           5       0.34      0.38      0.36       312

    accuracy                           0.44      1682
   macro avg       0.43      0.42      0.43      1682
weighted avg       0.46      0.44      0.44      1682



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.01)

In [ ]:
gb.fit(X_train , y_train.values.ravel())

GradientBoostingClassifier(learning_rate=0.01, n_estimators=500)

In [ ]:
gbp = gb.predict(X_test)

In [ ]:
print(classification_report(gbp, y_test))

              precision    recall  f1-score   support

           1       0.41      0.45      0.43       313
           2       0.43      0.50      0.46       272
           3       0.62      0.63      0.62       326
           4       0.85      0.68      0.76       424
           5       0.46      0.46      0.46       347

    accuracy                           0.55      1682
   macro avg       0.55      0.55      0.55      1682
weighted avg       0.58      0.55      0.56      1682



In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dm = DecisionTreeClassifier()

In [ ]:
dm.fit(X_train, y_train.values.ravel())

DecisionTreeClassifier()

In [ ]:
dy = dm.predict(X_test)

In [ ]:
print(classification_report(dy, y_test))

              precision    recall  f1-score   support

           1       0.43      0.42      0.42       349
           2       0.65      0.60      0.62       347
           3       0.48      0.53      0.50       301
           4       0.87      0.81      0.84       364
           5       0.47      0.51      0.49       321

    accuracy                           0.58      1682
   macro avg       0.58      0.57      0.58      1682
weighted avg       0.59      0.58      0.58      1682

